In [1]:
## Requires serparate venv for sb3

import sys, os
parent_dir = os.getcwd()
path = os.path.dirname(parent_dir)
sys.path.append(path)

from stable_baselines3.common.env_checker import check_env
from gym import spaces, wrappers
from gym_homer.envs.test_env import HomerEnv
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(path+"/test_env_data.csv")
data.loc[:,'time']=data.apply(lambda x: np.array([x.time_x, x.time_y], dtype=np.float32), axis=1)
data.loc[:,'month']=data.apply(lambda x: np.array([x.month_x, x.month_y], dtype=np.float32), axis=1)
data.loc[:,'region']=data.apply(lambda x: np.array([x.region_1, x.region_2, x.region_3], dtype=np.int8), axis=1)
data.loc[:,'weekend']=data.apply(lambda x: np.array(x.weekend, dtype=np.int8), axis=1)

In [3]:
data

,time_x,time_y,weekend,month_x,month_y,region_1,region_2,region_3,solar,home,import_tariff,export_tariff,time,month,region
0,0.000,1.000,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[0.0, 1.0]","[0.0, 1.0]","[0, 0, 0]"
1,0.500,0.866,0,0.0,1.0,0,0,0,1,1,0.05,0.01,"[0.5, 0.866]","[0.0, 1.0]","[0, 0, 0]"
2,0.866,0.500,0,0.0,1.0,0,0,0,1,1,0.05,0.01,"[0.866, 0.5]","[0.0, 1.0]","[0, 0, 0]"
3,1.000,0.000,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[1.0, 0.0]","[0.0, 1.0]","[0, 0, 0]"
4,0.866,-0.500,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[0.866, -0.5]","[0.0, 1.0]","[0, 0, 0]"
5,0.500,-0.866,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[0.5, -0.866]","[0.0, 1.0]","[0, 0, 0]"
6,0.000,-1.000,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[0.0, -1.0]","[0.0, 1.0]","[0, 0, 0]"
7,-0.500,-0.866,0,0.0,1.0,0,0,0,0,1,0.05,1.00,"[-0.5, -0.866]","[0.0, 1.0]","[0, 0, 0]"
8,-0.866,-0.500,0,0.0,1.0,0,0,0,0,1,0.05,1.00,"[-0.866, -0.5]","[0.0, 1.0]","[0, 0, 0]"
9,-1.000,-0.000,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[-1.0, -0.0]","[0.0, 1.0]","[0, 0, 0]"


In [4]:
data.solar.sum() * 1 - (data.home.sum() * 0.05)

1.4

In [ ]:
import gym
env = gym.make('gym_homer/HomerEnv-v0', data=data)

In [16]:
check_env(env, warn=True)

In [7]:
wrapped_env = wrappers.FlattenObservation(env)
print(wrapped_env.reset())

(array([ 0.01,  0.05,  1.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,
        0.  ,  0.  , -0.  ,  0.  ,  1.  ,  0.  ], dtype=float32), {'reward': 0.05000000074505806, 'net': array([1.], dtype=float32), 'action': 2, 'bat_output': 0})


In [17]:
from stable_baselines3 import PPO

model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

obs = env.reset()
for i in range(10):
    action, _states = model.predict(obs, deterministic=True)
    obs[0], reward, done, info = env.step(action)
    #env.render()
    if done:
        obs = wrapped_env.reset()
        

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 11       |
|    ep_rew_mean     | 0.537    |
| time/              |          |
|    fps             | 4611     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11          |
|    ep_rew_mean          | 0.589       |
| time/                   |             |
|    fps                  | 3425        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.020456048 |
|    clip_fraction        | 0.274       |
|    clip_range           | 0.2         |
|    entropy_loss   

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [18]:
for key, subspace in env.observation_space.spaces.items():
    if obs[0][key].shape != subspace.shape:
        print('!')

In [20]:
for key, subspace in env.observation_space.spaces.items():
    if obs[key].shape != subspace.shape:
        print('!')

TypeError: tuple indices must be integers or slices, not str

In [23]:
obs

({'solar': array([-0.], dtype=float32),
  'loads': array([1.], dtype=float32),
  'soc': array([0.], dtype=float32),
  'max_discharge': array([0.], dtype=float32),
  'max_charge': array([1.], dtype=float32),
  'region': array([0, 0, 0], dtype=int8),
  'import_tariff': array([0.05], dtype=float32),
  'export_tariff': array([0.01], dtype=float32),
  'time': array([0., 1.], dtype=float32),
  'weekend': array([array(0, dtype=int8)], dtype=object),
  'month': array([0., 1.], dtype=float32)},
 {'reward': 0.05000000074505806,
  'net': array([1.], dtype=float32),
  'action': 2,
  'bat_output': 0})

In [22]:
env.observation_space.spaces

OrderedDict([('export_tariff', Box(0.0, inf, (1,), float32)),
             ('import_tariff', Box(0.0, inf, (1,), float32)),
             ('loads', Box(0.0, inf, (1,), float32)),
             ('max_charge', Box(0.0, inf, (1,), float32)),
             ('max_discharge', Box(-inf, 0.0, (1,), float32)),
             ('month', Box(0.0, 1.0, (2,), float32)),
             ('region', MultiBinary(3)),
             ('soc', Box(0.0, inf, (1,), float32)),
             ('solar', Box(-inf, 0.0, (1,), float32)),
             ('time', Box(0.0, 1.0, (2,), float32)),
             ('weekend', MultiBinary(1))])